## 01. 檢查 GPU

In [ ]:
! nvidia-smi

## 02. 使用 git 來下載 Darknet (AlexeyAB 版本)

In [ ]:
! git clone https://github.com/AlexeyAB/darknet.git

## 03. head 來查看 Makefile 的參數

In [ ]:
! head darknet/Makefile

## 04. 參數說明
- GPU=1 調用 GPU CUDA
- CUDNN=1 調用 cuDNN v5-v7 加速訓練 
- CUDNN_HALF=1 調用 Tensor Cores (適用於 Titan V / Tesla V100 / DGX-2 and later) 偵測加速 3 倍, 訓練加速 2 倍
- OPENCV=1 調用 OpenCV 3.x/2.4.x - 用於攝影機偵測用

## 05. sad 來逐步修改 Makefile 內的參數

In [ ]:
! sed -i "s/GPU=0/GPU=1/g" /content/darknet/Makefile
! sed -i "s/CUDNN=0/CUDNN=1/g" /content/darknet/Makefile
! sed -i "s/OPENCV=0/OPENCV=1/g" /content/darknet/Makefile

In [ ]:
! head darknet/Makefile

## 06. 編譯 YOLOv4

In [ ]:
! cd darknet; make

## 使用 YOLOv4

In [ ]:
# 位置更正至darknet資料夾中
! cd darknet ; ./darknet 

## 下載權重
- Yolo v4 
- Yolo Tiny

In [ ]:
# yolo v4
! gdown https://drive.google.com/uc?id=1cewMfusmPjYWbrnuJRuKhPMwRe_b9PaT 

# yolo v4 Tiny
! gdown https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

# 連結至雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 雲端硬碟資料進行複製

In [ ]:
# 複製影像與標籤資料的壓縮檔
! cp "/content/drive/MyDrive/Dataset/Car_License_Plate/positive.zip" /content/

In [ ]:
# 解壓縮車牌影像資料
! unzip /content/positive.zip > data_unzip.log

## 使用YOLOv4偵測與辨識物件

In [ ]:
#給予設定檔、權重與影片來作預測
#! cd darknet; ./darknet detect /content/darknet/cfg/yolov4.cfg  /content/yolov4.weights /content/darknet/data/person.jpg

#自己雲端內的影像
! cd darknet; ./darknet detect /content/darknet/cfg/yolov4-tiny.cfg  /content/yolov4-tiny.weights /content/rawdata/bmpraw051.bmp

In [ ]:
from PIL import Image
Image.open('/content/darknet/predictions.jpg')

In [ ]:
import os
import shutil

path = "/content/label"
folder = os.path.exists(path) #判斷目錄是否存在

#如果不存在，則建立新目錄
if not folder: 
  os.makedirs(path) 

#如果目錄已存在，則刪除再建立
else:
  shutil.rmtree(path)
  os.makedirs(path) 

In [ ]:
import cv2

with open("/content/info.txt", mode="r") as f:
  text = f.read()
  text = text.split()
  for i in range(int(len(text)/6)):
    new_text = ""

    img = cv2.imread(text[i*6])
    height, width, channels = img.shape

    label = str(0) # 原本標註檔text[i*6+1]為1，但是yolo標籤是以0開始計算，所以這裡設為0
    x = int(text[i*6+2])
    y = int(text[i*6+3])
    w = int(text[i*6+4])
    h = int(text[i*6+5])
    ncx = (x+w/2)/width
    ncy = (y+h/2)/height
    nw = w/width
    nh = h/height
    new_text += label + " " + str(ncx) + " " + str(ncy) + " " + str(nw) + " " + str(nh)
    
    with open("/content/label/"+ text[i*6][8:17] +".txt", mode="w") as fn:
      print(new_text, file=fn)

## 測試正規化後的標籤轉化後繪製於影像上

In [ ]:
import matplotlib.pyplot as plt

img = cv2.imread("/content/rawdata/bmpraw001.bmp")
height, width, channels = img.shape

with open("/content/label/bmpraw001.txt", mode="r") as f:
  text = f.read()
  text = text.split()

  w = int(float(text[3])*width)
  h = int(float(text[4])*height)
  x = int(float(text[1])*width-w/2)
  y = int(float(text[2])*height-h/2)

cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 1)

plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# 建立設定檔空資料夾
! rm -rf /content/cfg_mask
! mkdir /content/cfg_mask

# 建立存放標籤與影像資料夾
! rm -rf /content/yolo
! mkdir /content/yolo

# 將影像與標籤複製於yolo資料夾中
! cp /content/label/* /content/yolo
! cp /content/rawdata/* /content/yolo

In [ ]:
# 將非".txt"檔案的路徑字串存放於datasets
datasets = ['/content/yolo/'+ f for f in os.listdir('/content/yolo/') if not f.endswith('.txt')]
len(datasets) * 0.8

In [ ]:
with open('/content/cfg_mask/train.txt', 'w') as f:
    f.write('\n'.join(datasets[0:400]))

In [ ]:
with open('/content/cfg_mask/test.txt', 'w') as f:
    f.write('\n'.join(datasets[400:]))

## 尋找設定檔(.data .names .cfg)與建立自己的設定檔
1. 從darknet複製的原始.data、.names、.cfg檔案，存在cfg_mask
2. 關於`.data`：
       
        class = 1
        train = /content/cfg_mask/train.txt
        valid = /content/cfg_mask/test.txt
        names = /content/cfg_mask/coco.names
        backup = /content/cfg_mask
其中classes為種類數，train為訓練資料的詳細位置，valid為驗證資料的詳細位置，names為.names的詳細位置，backup為權重存在的位置

3. .names檔案，更改為你的標籤名稱：
        License Plate
4. 關於`.cfg`：
* 第1-7行：

        [net]
        # Testing
        # batch=1
        # subdivisions=1
        # Training
        batch=64
        subdivisions=1
由於是進行訓練，這裡不需要修改。訓練過程中可能出現CUDA out of memory的提示，可將這裡的subdivisions增大，如16,32或64，但是數值越大耗时越长，因此需要權衡一下。

* 第8-9行將608修改為416 (Tiny預設為416)：

        width=416
        height=416
注意：這裡也可不改，如果原始的數值608可能會導致CUDA out of memory的提示，而且這裡的數值必須是32的倍數，這裡也是數值越大耗时越長。

* 第21行的參數max_batches也要修改，原始值為500500 (Tiny預設為2000200)，max_batches = classes*2000，但是max_batches不要低於訓練的影像張數，這裡只訓練1類，因此max_batches = 2000。

* 第23行的參數steps=4800,5400 (Tiny預設為160000, 180000)，這兩個數值分别為max_batches的80%和90%。

* 按Ctrl+F鍵，搜索“classes”，一共有2或3處，先定位到第一處，將classes=80改為classes=1，並將classes前面最近的filters修改为18，計算方式為（classes+5）*3=18。按照上面的步驟同樣修改第2(與3)處的classes與filters。


In [ ]:
#從darknet複製的原始.data、.names、.cfg檔案，存在cfg_mask
! cp "/content/darknet/cfg/coco.data" cfg_mask/
! cp "/content/darknet/cfg/coco.names" cfg_mask/
! cp "/content/darknet/cfg/yolov4-tiny.cfg" cfg_mask/
#! cp "/content/darknet/cfg/yolov4.cfg" cfg_mask/

In [ ]:
# 下載預訓練模型
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# 訓練模型

In [ ]:
!./darknet/darknet detector train cfg_mask/coco.data cfg_mask/yolov4-tiny.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# 將最後權重與設定檔複製於雲端硬碟中
! cp /content/cfg_mask/yolov4-tiny_final.weights /content/drive/MyDrive
! cp /content/cfg_mask/yolov4-tiny.cfg /content/drive/MyDrive